In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
train_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
test_data = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
test_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [4]:
print(train_data.columns)
print(test_data.columns)

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')
Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name'],
      dtype='object')


In [5]:
# trying to find if any significant differences in survival rate between home planets

home_planet = train_data.loc[train_data.HomePlanet == 'Europa']['Transported']
print(f"Europe survival rate: {sum(home_planet)/len(home_planet)}")

earth = train_data.loc[train_data.HomePlanet == 'Earth']['Transported']
print(f"Earth survival rate: {sum(earth)/len(earth)}")

Europe survival rate: 0.65884561238855
Earth survival rate: 0.42394611038678837


In [6]:
from sklearn.impute import SimpleImputer

# check and deal with NaN values
# print("Train data NaN values: \n", train_data.isna().sum(), end="\n\n")

features = ['CryoSleep', 'Age', 'VIP', 'HomePlanet']

y = train_data.Transported
train_data.drop(['Transported'], axis=1, inplace=True)

imputer = SimpleImputer(strategy='most_frequent')
imputed_train_data = pd.DataFrame(imputer.fit_transform(train_data))
imputed_test_data = pd.DataFrame(imputer.transform(test_data))

# add columns back
imputed_train_data.columns = train_data.columns
imputed_test_data.columns = test_data.columns

In [7]:
from sklearn.ensemble import RandomForestClassifier

X = pd.get_dummies(imputed_train_data[features])
X_test = pd.get_dummies(imputed_test_data[features])

# remove column that doesnt exist in X_test but is present in X
X.drop(['Age_76.0'], axis=1, inplace=True)

model = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=1)
model.fit(X, y,)
predictions = model.predict(X_test)

predictions

array([ True, False,  True, ...,  True, False,  True])

In [8]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Transported': predictions})
output.to_csv('spaceship_titanic_submission.csv', index=False)
print("Saved successfully")

Saved successfully


In [9]:
output.describe()

,PassengerId,Transported
count,4277,4277
unique,4277,2
top,0013_01,False
freq,1,2647
